In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:

!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U huggingface_hub

!pip install -q -U langchain

!pip install -q -U pypdf

!pip install -q -U sentence-transformers

!pip install -q -U faiss-gpu
!pip install -q -U chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4


In [ ]:
import accelerate

In [ ]:



!pip -qqq install bitsandbytes accelerate

In [ ]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(model_path)


save_directory = "/content/drive/MyDrive/llm_chatbot"



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:

import json

pipeline_config = {
    "task": "text-generation",
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.eos_token_id,
    "repetition_penalty": 1.1,
    "return_full_text": False,
    "max_new_tokens": 300,
    "temperature": 0.3,
    "do_sample": True
}


pipeline_config_path = "/content/drive/MyDrive/llm_chatbot/pipeline_config.json"

with open(pipeline_config_path, 'w') as f:
    json.dump(pipeline_config, f)

print(f"Pipeline configuration saved to {pipeline_config_path}")



save_directory = "/content/drive/MyDrive/llm_chatbot"



pipeline_config_path = "/content/drive/MyDrive/llm_chatbot/pipeline_config.json"
with open(pipeline_config_path, 'r') as f:
    pipeline_config = json.load(f)


text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task=pipeline_config["task"],
    eos_token_id=pipeline_config["eos_token_id"],
    pad_token_id=pipeline_config["pad_token_id"],
    repetition_penalty=pipeline_config["repetition_penalty"],
    return_full_text=pipeline_config["return_full_text"],
    max_new_tokens=pipeline_config["max_new_tokens"],
    temperature=pipeline_config["temperature"],
    do_sample=pipeline_config["do_sample"]
)
ne
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("Pipeline recreated successfully.")




Pipeline configuration saved to /content/drive/MyDrive/llm_chatbot/pipeline_config.json
Pipeline recreated successfully.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


# **Using CSV File with LangChain CSVLoader**

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader


import pandas as pd
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/recipe generation/Cleaned_Indian_Food_Dataset.csv')
from langchain.schema.document import Document
text = ""
for ind in df.index:
    text += f"{df['TranslatedRecipeName'][ind]}  {df['TranslatedIngredients'][ind]}  {df['TotalTimeInMins'][ind]} {df['Cuisine'][ind]} {df['TranslatedInstructions'][ind]} {df['Cleaned-Ingredients'][ind]} {df['Ingredient-count'][ind]}\n#####\n"

documents = Document(page_content=text, metadata={"source": "local"})


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents([documents])

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f0f0f0;
        }
        #chat-container {
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        #chat {
            width: 300px;
            height: 400px;
            overflow-y: scroll;
            border: 1px solid #ccc;
            padding: 10px;
            margin-bottom: 10px;
        }
        #input {
            width: 100%;
            padding: 10px;
            border-radius: 5px;
            border: 1px solid #ccc;
        }
        .message {
            margin: 5px 0;
            padding: 5px 10px;
            border-radius: 5px;
        }
        .user {
            background: #d1e7dd;
            align-self: flex-end;
        }
        .bot {
            background: #f8d7da;
            align-self: flex-start;
        }
    </style>
</head>
<body>
    <div id="chat-container">
        <div id="chat"></div>
        <input type="text" id="input" placeholder="Type a message..." onkeydown="if(event.key === 'Enter') sendMessage()">
    </div>
    <script>
        const apiUrl = https://32f0-34-124-249-205.ngrok-free.app/chatbot/'; // Replace with your actual ngrok URL

        async function sendMessage() {
            const inputElement = document.getElementById('input');
            const chatElement = document.getElementById('chat');
            const message = inputElement.value.trim();

            if (!message) return;

            // Display user message
            const userMessageElement = document.createElement('div');
            userMessageElement.className = 'message user';
            userMessageElement.textContent = message;
            chatElement.appendChild(userMessageElement);
            chatElement.scrollTop = chatElement.scrollHeight;

            // Clear input
            inputElement.value = '';

            try {
                // Send message to the server
                const response = await fetch(apiUrl, {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ text: message }),
                });
                const data = await response.json();

                // Display bot response
                const botMessageElement = document.createElement('div');
                botMessageElement.className = 'message bot';
                botMessageElement.textContent = data.response;
                chatElement.appendChild(botMessageElement);
                chatElement.scrollTop = chatElement.scrollHeight;
            } catch (error) {
                console.error('Error:', error);
            }
        }
    </script>
</body>
</html>


In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio chainlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.2 MB/s eta 0:00:00
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.25.0-py3-none-any.whl (18 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.25.0-py3-none-any.whl (17 kB)
  Using cached opentelemetry_proto-1.25.0-py3-none-any.whl (52 kB)
  Using cached opentelemetry_sdk-1.25.0-py3-none-any.whl (107 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl (59 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.8 

In [ ]:
# Expose the app via ngrok
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import json
import os
ngrok.set_auth_token("2h2ClSz9xtjDrqOqKaMtoNijQsZ_4pUoTGb5N6fzRcX3jNLfz")
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the app


Public URL: NgrokTunnel: "https://6a60-34-125-83-204.ngrok-free.app" -> "http://localhost:8000"


In [ ]:

app = FastAPI()

class Prompt(BaseModel):
    text: str
    chat_history: list

@app.post("/chatbot/")
async def get_response(prompt: Prompt):
    query = prompt.text
    chat_history = prompt.chat_history
    result = qa_chain({'question': query, 'chat_history': chat_history})
    chat_history.append((query, result['answer']))
    return {"response": result['answer'], "chat_history": chat_history}

# Serve the HTML UI
@app.get("/", response_class=HTMLResponse)
async def serve_ui():
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Chatbot</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                background-color: #f0f0f0;
            }
            #chat-container {
                background: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            #chat {
                width: 300px;
                height: 400px;
                overflow-y: scroll;
                border: 1px solid #ccc;
                padding: 10px;
                margin-bottom: 10px;
            }
            #input {
                width: 100%;
                padding: 10px;
                border-radius: 5px;
                border: 1px solid #ccc;
            }
            .message {
                margin: 5px 0;
                padding: 5px 10px;
                border-radius: 5px;
            }
            .user {
                background: #d1e7dd;
                align-self: flex-end;
            }
            .bot {
                background: #f8d7da;
                align-self: flex-start;
            }
        </style>
    </head>
    <body>
        <div id="chat-container">
            <div id="chat"></div>
            <input type="text" id="input" placeholder="Type a message..." onkeydown="if(event.key === 'Enter') sendMessage()">
        </div>
        <script>
            const apiUrl = 'https://6a60-34-125-83-204.ngrok-free.app/chatbot/'; // Replace with your actual ngrok URL

            async function sendMessage() {
                const inputElement = document.getElementById('input');
                const chatElement = document.getElementById('chat');
                const message = inputElement.value.trim();
                let chatHistory = [];

                if (!message) return;

                // Display user message
                const userMessageElement = document.createElement('div');
                userMessageElement.className = 'message user';
                userMessageElement.textContent = message;
                chatElement.appendChild(userMessageElement);
                chatElement.scrollTop = chatElement.scrollHeight;

                // Clear input
                inputElement.value = '';

                try {
                    // Send message to the server
                    const response = await fetch(apiUrl, {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                        },
                        body: JSON.stringify({ text: message, chat_history: chatHistory }),
                    });
                    const data = await response.json();

                    // Update chat history
                    chatHistory = data.chat_history;

                    // Display bot response
                    const botMessageElement = document.createElement('div');
                    botMessageElement.className = 'message bot';
                    botMessageElement.textContent = data.response;
                    chatElement.appendChild(botMessageElement);
                    chatElement.scrollTop = chatElement.scrollHeight;
                } catch (error) {
                    console.error('Error:', error);
                }
            }
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)




In [ ]:
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [560]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "POST /chatbot/ HTTP/1.1" 200 OK
INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "POST /chatbot/ HTTP/1.1" 200 OK
INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "POST /chatbot/ HTTP/1.1" 200 OK
INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "POST /chatbot/ HTTP/1.1" 200 OK
INFO:     2405:201:e006:3065:5fa:cb86:b678:7903:0 - "POST /chatbot/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [560]


SyntaxError: invalid decimal literal (<ipython-input-6-a278a430dccb>, line 1)

In [ ]:

import sys
chat_history = []
query = input('Prompt: ')
result = qa_chain({'question': query, 'chat_history': chat_history})
retrieved_docs = result['source_documents']
print(retrieved_docs)
print(chat_history,"ss")

    # Check if any relevant documents were retrieved
ans=result['answer']

if not retrieved_docs:
      ans = "The information is not available in the dataset."



print('Answer: ' + ans + '\n')
chat_history.append((query, ans))

Prompt: go trhough the process of making pudhina chutney
[Document(page_content='#\nPudina Chutney Recipe  1 teaspoon White Urad Dal (Split),2 teaspoons Tamarind Paste,1/2 teaspoon Mustard seeds,3 Dry Red Chillies,Salt - to taste,1/2 tablespoon Jaggery,2 cups Mint Leaves (Pudina) - cleaned,5 to 6 Curry leaves,1 teaspoon Coriander (Dhania) Seeds,2 Dry Red Chillies,1/4 teaspoon Turmeric powder (Haldi),1 teaspoon Sunflower Oil,1/4 teaspoon Asafoetida (hing),1/3 cup Fresh coconut - grated,2 teaspoons White Urad Dal (Split)  30 North Indian Recipes To begin making the Pudina Chutney Recipe, first dry the washed mint leaves on a kitchen towel.Heat oil in a wide pan coriander seeds and  urad dal and saute till the urad dal turns golden brown in color.\nOnce done add the dry chillies and saute till they turn crisp, then add coconut, turmeric powder and mint leaves.\nSaute till the water from the mint leaves dry up.\nTurn off the heat and allow to cool down.Once cooled add this into a small mix

#**Using CSV file with Pandas dataframe**

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 36.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
bigframes 0.19.2 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.0 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.0 which is incompatible.


In [ ]:
#print(text)

Prompt: What is Carbon dioxide emission of Australia?
Answer:  To calculate carbon dioxide emissions for a country, we need to look at the data for the year in which we are interested and add up the values in the column labeled "CO2 Emissions". For example, to find the carbon dioxide emissions for Australia in 2014, we would add up the values in the row labeled "Australia" for the year 2014. The result would be 522397.090711347. We can do this for any year by looking at the corresponding row in the table and adding up the values in the "CO2 Emissions" column.

